# Structured Outputs For Reasoning Models

When working with reasoning models that use special tokens like `<think>...</think>` to denote reasoning sections, you might want to allow free-form text within these sections while still enforcing grammar constraints on the rest of the output.

SGLang provides a feature to disable grammar restrictions within reasoning sections. This is particularly useful for models that need to perform complex reasoning steps before providing a structured output.

To enable this feature, use the `--reasoning-parser` flag which decide the think_end_token, such as `</think>`, when launching the server. You can also specify the reasoning parser using the `--reasoning-parser` flag.

## Supported Models

Currently, SGLang supports the following reasoning models:
- [DeepSeek R1 series](https://huggingface.co/collections/deepseek-ai/deepseek-r1-678e1e131c0169c0bc89728d): The reasoning content is wrapped with `<think>` and `</think>` tags.
- [QwQ](https://huggingface.co/Qwen/QwQ-32B): The reasoning content is wrapped with `<think>` and `</think>` tags.


## Usage

## OpenAI Compatible API

Specify the `--grammar-backend`, `--reasoning-parser` option.

In [1]:
import openai
import os

from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

os.environ["TOKENIZERS_PARALLELISM"] = "false"


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path deepseek-ai/DeepSeek-R1-Distill-Qwen-7B --host 0.0.0.0 --reasoning-parser deepseek-r1"
)

wait_for_server(f"http://localhost:{port}")
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

W0901 06:37:08.501000 2827727 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0901 06:37:08.501000 2827727 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


`torch_dtype` is deprecated! Use `dtype` instead!
[2025-09-01 06:37:09] server_args=ServerArgs(model_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=37911, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_

[2025-09-01 06:37:10] Using default HuggingFace chat template with detected content format: string


W0901 06:37:19.134000 2828113 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0901 06:37:19.134000 2828113 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


W0901 06:37:19.345000 2828112 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0901 06:37:19.345000 2828112 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


...

[2025-09-01 06:37:29] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=62.64 GB


[2025-09-01 06:37:30] INFO:     Started server process [2827727]
[2025-09-01 06:37:30] INFO:     Waiting for application startup.
[2025-09-01 06:37:30] INFO:     Application startup complete.
[2025-09-01 06:37:30] INFO:     Uvicorn running on http://0.0.0.0:37911 (Press CTRL+C to quit)


[2025-09-01 06:37:30] INFO:     127.0.0.1:59020 - "GET /v1/models HTTP/1.1" 200 OK


[2025-09-01 06:37:31] INFO:     127.0.0.1:59034 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-09-01 06:37:31] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-09-01 06:37:32] INFO:     127.0.0.1:59048 - "POST /generate HTTP/1.1" 200 OK
[2025-09-01 06:37:32] The server is fired up and ready to roll!


### JSON

you can directly define a JSON schema or use [Pydantic](https://docs.pydantic.dev/latest/) to define and validate the response.

**Using Pydantic**

In [2]:
from pydantic import BaseModel, Field


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "foo",
            # convert the pydantic model to json schema
            "schema": CapitalInfo.model_json_schema(),
        },
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-09-01 06:37:36] Prefill batch. #new-seq: 1, #new-token: 21, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-09-01 06:37:37] Decode batch. #running-req: 1, #token: 55, token usage: 0.00, cuda graph: True, gen throughput (token/s): 5.19, #queue-req: 0, 


[2025-09-01 06:37:37] Decode batch. #running-req: 1, #token: 95, token usage: 0.00, cuda graph: True, gen throughput (token/s): 165.56, #queue-req: 0, 


[2025-09-01 06:37:37] Decode batch. #running-req: 1, #token: 135, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.27, #queue-req: 0, 


[2025-09-01 06:37:38] Decode batch. #running-req: 1, #token: 175, token usage: 0.01, cuda graph: True, gen throughput (token/s): 163.78, #queue-req: 0, 


...

[2025-09-01 06:37:38] Decode batch. #running-req: 1, #token: 295, token usage: 0.01, cuda graph: True, gen throughput (token/s): 163.67, #queue-req: 0, 


[2025-09-01 06:37:39] Decode batch. #running-req: 1, #token: 335, token usage: 0.02, cuda graph: True, gen throughput (token/s): 163.70, #queue-req: 0, 


[2025-09-01 06:37:39] Decode batch. #running-req: 1, #token: 375, token usage: 0.02, cuda graph: True, gen throughput (token/s): 163.66, #queue-req: 0, 


[2025-09-01 06:37:39] Decode batch. #running-req: 1, #token: 415, token usage: 0.02, cuda graph: True, gen throughput (token/s): 163.26, #queue-req: 0, 


[2025-09-01 06:37:39] Decode batch. #running-req: 1, #token: 455, token usage: 0.02, cuda graph: True, gen throughput (token/s): 163.65, #queue-req: 0, 
[2025-09-01 06:37:39] INFO:     127.0.0.1:43550 - "POST /v1/chat/completions HTTP/1.1" 200 OK


**JSON Schema Directly**


In [3]:
import json

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "foo", "schema": json.loads(json_schema)},
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-09-01 06:37:39] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 21, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-09-01 06:37:40] Decode batch. #running-req: 1, #token: 56, token usage: 0.00, cuda graph: True, gen throughput (token/s): 149.86, #queue-req: 0, 


[2025-09-01 06:37:40] Decode batch. #running-req: 1, #token: 96, token usage: 0.00, cuda graph: True, gen throughput (token/s): 169.23, #queue-req: 0, 


[2025-09-01 06:37:40] Decode batch. #running-req: 1, #token: 136, token usage: 0.01, cuda graph: True, gen throughput (token/s): 168.83, #queue-req: 0, 


[2025-09-01 06:37:40] Decode batch. #running-req: 1, #token: 176, token usage: 0.01, cuda graph: True, gen throughput (token/s): 167.26, #queue-req: 0, 


[2025-09-01 06:37:41] Decode batch. #running-req: 1, #token: 216, token usage: 0.01, cuda graph: True, gen throughput (token/s): 166.97, #queue-req: 0, 


[2025-09-01 06:37:41] Decode batch. #running-req: 1, #token: 256, token usage: 0.01, cuda graph: True, gen throughput (token/s): 167.26, #queue-req: 0, 


[2025-09-01 06:37:41] Decode batch. #running-req: 1, #token: 296, token usage: 0.01, cuda graph: True, gen throughput (token/s): 167.00, #queue-req: 0, 


[2025-09-01 06:37:41] Decode batch. #running-req: 1, #token: 336, token usage: 0.02, cuda graph: True, gen throughput (token/s): 164.22, #queue-req: 0, 


[2025-09-01 06:37:42] Decode batch. #running-req: 1, #token: 376, token usage: 0.02, cuda graph: True, gen throughput (token/s): 160.91, #queue-req: 0, 
[2025-09-01 06:37:42] INFO:     127.0.0.1:43550 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### EBNF

In [4]:
ebnf_grammar = """
root ::= city | description
city ::= "London" | "Paris" | "Berlin" | "Rome"
description ::= city " is " status
status ::= "the capital of " country
country ::= "England" | "France" | "Germany" | "Italy"
"""

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "system", "content": "You are a helpful geography bot."},
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"ebnf": ebnf_grammar},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-09-01 06:37:42] Prefill batch. #new-seq: 1, #new-token: 28, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-09-01 06:37:42] Decode batch. #running-req: 1, #token: 61, token usage: 0.00, cuda graph: True, gen throughput (token/s): 147.11, #queue-req: 0, 


[2025-09-01 06:37:42] Decode batch. #running-req: 1, #token: 101, token usage: 0.00, cuda graph: True, gen throughput (token/s): 165.54, #queue-req: 0, 


[2025-09-01 06:37:42] Decode batch. #running-req: 1, #token: 141, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.01, #queue-req: 0, 


[2025-09-01 06:37:43] Decode batch. #running-req: 1, #token: 181, token usage: 0.01, cuda graph: True, gen throughput (token/s): 163.85, #queue-req: 0, 


...

[2025-09-01 06:37:45] Decode batch. #running-req: 1, #token: 581, token usage: 0.03, cuda graph: True, gen throughput (token/s): 153.33, #queue-req: 0, 


[2025-09-01 06:37:45] Decode batch. #running-req: 1, #token: 621, token usage: 0.03, cuda graph: True, gen throughput (token/s): 158.40, #queue-req: 0, 


[2025-09-01 06:37:46] Decode batch. #running-req: 1, #token: 661, token usage: 0.03, cuda graph: True, gen throughput (token/s): 163.80, #queue-req: 0, 


[2025-09-01 06:37:46] Decode batch. #running-req: 1, #token: 701, token usage: 0.03, cuda graph: True, gen throughput (token/s): 164.04, #queue-req: 0, 


[2025-09-01 06:37:46] Decode batch. #running-req: 1, #token: 741, token usage: 0.04, cuda graph: True, gen throughput (token/s): 164.92, #queue-req: 0, 
[2025-09-01 06:37:46] INFO:     127.0.0.1:43550 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Regular expression

In [5]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "assistant", "content": "What is the capital of France?"},
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"regex": "(Paris|London)"},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-09-01 06:37:46] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-09-01 06:37:46] Decode batch. #running-req: 1, #token: 31, token usage: 0.00, cuda graph: True, gen throughput (token/s): 147.55, #queue-req: 0, 


[2025-09-01 06:37:47] Decode batch. #running-req: 1, #token: 71, token usage: 0.00, cuda graph: True, gen throughput (token/s): 168.81, #queue-req: 0, 


[2025-09-01 06:37:47] Decode batch. #running-req: 1, #token: 111, token usage: 0.01, cuda graph: True, gen throughput (token/s): 169.15, #queue-req: 0, 
[2025-09-01 06:37:47] INFO:     127.0.0.1:43550 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Structural Tag

In [6]:
tool_get_current_weather = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to find the weather for, e.g. 'San Francisco'",
                },
                "state": {
                    "type": "string",
                    "description": "the two-letter abbreviation for the state that the city is"
                    " in, e.g. 'CA' which would mean 'California'",
                },
                "unit": {
                    "type": "string",
                    "description": "The unit to fetch the temperature in",
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["city", "state", "unit"],
        },
    },
}

tool_get_current_date = {
    "type": "function",
    "function": {
        "name": "get_current_date",
        "description": "Get the current date and time for a given timezone",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "The timezone to fetch the current date and time for, e.g. 'America/New_York'",
                }
            },
            "required": ["timezone"],
        },
    },
}

schema_get_current_weather = tool_get_current_weather["function"]["parameters"]
schema_get_current_date = tool_get_current_date["function"]["parameters"]


def get_messages():
    return [
        {
            "role": "system",
            "content": f"""
# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available else fallback to brave_search
You have access to the following functions:
Use the function 'get_current_weather' to: Get the current weather in a given location
{tool_get_current_weather["function"]}
Use the function 'get_current_date' to: Get the current date and time for a given timezone
{tool_get_current_date["function"]}
If a you choose to call a function ONLY reply in the following format:
<{{start_tag}}={{function_name}}>{{parameters}}{{end_tag}}
where
start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`
Here is an example,
<function=example_function_name>{{"example_name": "example_value"}}</function>
Reminder:
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- Always add your sources when using search results to answer the user query
You are a helpful assistant.""",
        },
        {
            "role": "assistant",
            "content": "You are in New York. Please get the current date and time, and the weather.",
        },
    ]


messages = get_messages()

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=messages,
    response_format={
        "type": "structural_tag",
        "max_new_tokens": 2048,
        "structures": [
            {
                "begin": "<function=get_current_weather>",
                "schema": schema_get_current_weather,
                "end": "</function>",
            },
            {
                "begin": "<function=get_current_date>",
                "schema": schema_get_current_date,
                "end": "</function>",
            },
        ],
        "triggers": ["<function="],
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-09-01 06:37:47] Prefill batch. #new-seq: 1, #new-token: 472, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-09-01 06:37:47] Decode batch. #running-req: 1, #token: 491, token usage: 0.02, cuda graph: True, gen throughput (token/s): 113.83, #queue-req: 0, 


[2025-09-01 06:37:47] Decode batch. #running-req: 1, #token: 531, token usage: 0.03, cuda graph: True, gen throughput (token/s): 159.75, #queue-req: 0, 


[2025-09-01 06:37:48] Decode batch. #running-req: 1, #token: 571, token usage: 0.03, cuda graph: True, gen throughput (token/s): 164.46, #queue-req: 0, 


[2025-09-01 06:37:48] Decode batch. #running-req: 1, #token: 611, token usage: 0.03, cuda graph: True, gen throughput (token/s): 163.06, #queue-req: 0, 


[2025-09-01 06:37:48] Decode batch. #running-req: 1, #token: 651, token usage: 0.03, cuda graph: True, gen throughput (token/s): 161.12, #queue-req: 0, 


[2025-09-01 06:37:48] Decode batch. #running-req: 1, #token: 691, token usage: 0.03, cuda graph: True, gen throughput (token/s): 160.78, #queue-req: 0, 


[2025-09-01 06:37:49] Decode batch. #running-req: 1, #token: 731, token usage: 0.04, cuda graph: True, gen throughput (token/s): 155.10, #queue-req: 0, 


[2025-09-01 06:37:49] Decode batch. #running-req: 1, #token: 771, token usage: 0.04, cuda graph: True, gen throughput (token/s): 156.44, #queue-req: 0, 
[2025-09-01 06:37:49] INFO:     127.0.0.1:43550 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Native API and SGLang Runtime (SRT)

### JSON

**Using Pydantic**

In [7]:
import requests
from pydantic import BaseModel, Field
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


messages = [
    {
        "role": "assistant",
        "content": "Give me the information and population of the capital of France in the JSON format.",
    },
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
# Make API request
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json.dumps(CapitalInfo.model_json_schema()),
        },
    },
)
print(response.json())


reasoing_content = response.json()["text"].split("</think>")[0]
content = response.json()["text"].split("</think>")[1]
print_highlight(f"reasoing_content: {reasoing_content}\n\ncontent: {content}")

[2025-09-01 06:37:50] Prefill batch. #new-seq: 1, #new-token: 22, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-09-01 06:37:50] Decode batch. #running-req: 1, #token: 33, token usage: 0.00, cuda graph: True, gen throughput (token/s): 49.78, #queue-req: 0, 


[2025-09-01 06:37:50] Decode batch. #running-req: 1, #token: 73, token usage: 0.00, cuda graph: True, gen throughput (token/s): 166.40, #queue-req: 0, 


[2025-09-01 06:37:50] Decode batch. #running-req: 1, #token: 113, token usage: 0.01, cuda graph: True, gen throughput (token/s): 169.21, #queue-req: 0, 


[2025-09-01 06:37:50] Decode batch. #running-req: 1, #token: 153, token usage: 0.01, cuda graph: True, gen throughput (token/s): 168.09, #queue-req: 0, 


[2025-09-01 06:37:51] Decode batch. #running-req: 1, #token: 193, token usage: 0.01, cuda graph: True, gen throughput (token/s): 163.99, #queue-req: 0, 


...

[2025-09-01 06:37:51] Decode batch. #running-req: 1, #token: 273, token usage: 0.01, cuda graph: True, gen throughput (token/s): 153.57, #queue-req: 0, 


[2025-09-01 06:37:52] Decode batch. #running-req: 1, #token: 313, token usage: 0.02, cuda graph: True, gen throughput (token/s): 135.55, #queue-req: 0, 


[2025-09-01 06:37:52] Decode batch. #running-req: 1, #token: 353, token usage: 0.02, cuda graph: True, gen throughput (token/s): 139.84, #queue-req: 0, 


[2025-09-01 06:37:52] Decode batch. #running-req: 1, #token: 393, token usage: 0.02, cuda graph: True, gen throughput (token/s): 136.16, #queue-req: 0, 


[2025-09-01 06:37:52] INFO:     127.0.0.1:54642 - "POST /generate HTTP/1.1" 200 OK
{'text': 'Okay, so the user is asking for the information and population of the capital of France in JSON format. Let me break this down. First, I need to identify what the capital of France is. I know that Paris is the capital, so that\'s straightforward. \n\nNext, I need to find the population. I remember that Paris is a major city, so its population is quite large. I think it\'s over 3 million, but I\'m not exactly sure of the exact number. Maybe I should double-check that. \n\nWait, I recall that the population figure can vary depending on the source and the year. The user didn\'t specify a particular year, so I should probably go with the most recent estimate. I believe the population is around 3,500,000 as of 2023. \n\nNow, I need to structure this information into a JSON format. JSON typically uses key-value pairs, so I\'ll create an object with keys like "city", "population", and maybe "country" 

**JSON Schema Directly**

In [8]:
json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

# JSON
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json_schema,
        },
    },
)

print_highlight(response.json())

[2025-09-01 06:37:52] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 22, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-09-01 06:37:52] Decode batch. #running-req: 1, #token: 28, token usage: 0.00, cuda graph: True, gen throughput (token/s): 131.67, #queue-req: 0, 


[2025-09-01 06:37:53] Decode batch. #running-req: 1, #token: 68, token usage: 0.00, cuda graph: True, gen throughput (token/s): 145.46, #queue-req: 0, 


[2025-09-01 06:37:53] Decode batch. #running-req: 1, #token: 108, token usage: 0.01, cuda graph: True, gen throughput (token/s): 146.24, #queue-req: 0, 


[2025-09-01 06:37:53] Decode batch. #running-req: 1, #token: 148, token usage: 0.01, cuda graph: True, gen throughput (token/s): 144.29, #queue-req: 0, 


[2025-09-01 06:37:54] Decode batch. #running-req: 1, #token: 188, token usage: 0.01, cuda graph: True, gen throughput (token/s): 155.10, #queue-req: 0, 


...

[2025-09-01 06:37:54] Decode batch. #running-req: 1, #token: 268, token usage: 0.01, cuda graph: True, gen throughput (token/s): 155.88, #queue-req: 0, 


[2025-09-01 06:37:54] Decode batch. #running-req: 1, #token: 308, token usage: 0.02, cuda graph: True, gen throughput (token/s): 166.97, #queue-req: 0, 


[2025-09-01 06:37:54] Decode batch. #running-req: 1, #token: 348, token usage: 0.02, cuda graph: True, gen throughput (token/s): 164.51, #queue-req: 0, 


[2025-09-01 06:37:55] Decode batch. #running-req: 1, #token: 388, token usage: 0.02, cuda graph: True, gen throughput (token/s): 157.79, #queue-req: 0, 


[2025-09-01 06:37:55] Decode batch. #running-req: 1, #token: 1, token usage: 0.00, cuda graph: True, gen throughput (token/s): 166.70, #queue-req: 0, 
[2025-09-01 06:37:55] INFO:     127.0.0.1:54654 - "POST /generate HTTP/1.1" 200 OK


### EBNF

In [9]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Give me the information of the capital of France.",
        "sampling_params": {
            "max_new_tokens": 2048,
            "temperature": 0,
            "n": 3,
            "ebnf": (
                "root ::= city | description\n"
                'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
                'description ::= city " is " status\n'
                'status ::= "the capital of " country\n'
                'country ::= "England" | "France" | "Germany" | "Italy"'
            ),
        },
        "stream": False,
        "return_logprob": False,
    },
)

print(response.json())

[2025-09-01 06:37:55] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-09-01 06:37:55] Prefill batch. #new-seq: 3, #new-token: 3, #cached-token: 30, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-09-01 06:37:55] Decode batch. #running-req: 3, #token: 131, token usage: 0.01, cuda graph: True, gen throughput (token/s): 331.63, #queue-req: 0, 


[2025-09-01 06:37:56] Decode batch. #running-req: 3, #token: 251, token usage: 0.01, cuda graph: True, gen throughput (token/s): 484.70, #queue-req: 0, 


[2025-09-01 06:37:56] Decode batch. #running-req: 3, #token: 371, token usage: 0.02, cuda graph: True, gen throughput (token/s): 484.39, #queue-req: 0, 


[2025-09-01 06:37:56] Decode batch. #running-req: 3, #token: 491, token usage: 0.02, cuda graph: True, gen throughput (token/s): 479.16, #queue-req: 0, 


[2025-09-01 06:37:56] Decode batch. #running-req: 3, #token: 611, token usage: 0.03, cuda graph: True, gen throughput (token/s): 478.75, #queue-req: 0, 


[2025-09-01 06:37:57] Decode batch. #running-req: 3, #token: 731, token usage: 0.04, cuda graph: True, gen throughput (token/s): 478.44, #queue-req: 0, 
[2025-09-01 06:37:57] INFO:     127.0.0.1:44252 - "POST /generate HTTP/1.1" 200 OK
[{'text': "\nThe capital of France is Paris. It is located in the northern part of the country, along the Seine River. Paris is known for its rich history, art, and landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. It is a major city in France and has a significant cultural and economic impact.\n\nThe capital of France is Paris. It is located in the northern part of the country, along the Seine River. Paris is known for its rich history, art, and landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Damme Cathedral. It is a major city in France and has a significant cultural and economic impact.\n\nPlease provide the information in a clear and concise manner, using bullet points for the location and key landmarks

### Regular expression

In [10]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Paris is the capital of",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "regex": "(France|England)",
        },
    },
)
print(response.json())

[2025-09-01 06:37:57] Prefill batch. #new-seq: 1, #new-token: 5, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-09-01 06:37:57] Decode batch. #running-req: 1, #token: 32, token usage: 0.00, cuda graph: True, gen throughput (token/s): 251.49, #queue-req: 0, 


[2025-09-01 06:37:57] Decode batch. #running-req: 1, #token: 72, token usage: 0.00, cuda graph: True, gen throughput (token/s): 164.46, #queue-req: 0, 


[2025-09-01 06:37:57] Decode batch. #running-req: 1, #token: 112, token usage: 0.01, cuda graph: True, gen throughput (token/s): 145.11, #queue-req: 0, 


[2025-09-01 06:37:58] Decode batch. #running-req: 1, #token: 152, token usage: 0.01, cuda graph: True, gen throughput (token/s): 168.15, #queue-req: 0, 


...

[2025-09-01 06:38:09] Decode batch. #running-req: 1, #token: 1912, token usage: 0.09, cuda graph: True, gen throughput (token/s): 155.52, #queue-req: 0, 


[2025-09-01 06:38:09] Decode batch. #running-req: 1, #token: 1952, token usage: 0.10, cuda graph: True, gen throughput (token/s): 129.02, #queue-req: 0, 


[2025-09-01 06:38:10] Decode batch. #running-req: 1, #token: 1992, token usage: 0.10, cuda graph: True, gen throughput (token/s): 110.13, #queue-req: 0, 


[2025-09-01 06:38:10] Decode batch. #running-req: 1, #token: 2032, token usage: 0.10, cuda graph: True, gen throughput (token/s): 106.32, #queue-req: 0, 


[2025-09-01 06:38:10] INFO:     127.0.0.1:44260 - "POST /generate HTTP/1.1" 200 OK
{'text': ' France, and the \n\\( n \\)  \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \

### Structural Tag

In [11]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
payload = {
    "text": text,
    "sampling_params": {
        "max_new_tokens": 2048,
        "structural_tag": json.dumps(
            {
                "type": "structural_tag",
                "structures": [
                    {
                        "begin": "<function=get_current_weather>",
                        "schema": schema_get_current_weather,
                        "end": "</function>",
                    },
                    {
                        "begin": "<function=get_current_date>",
                        "schema": schema_get_current_date,
                        "end": "</function>",
                    },
                ],
                "triggers": ["<function="],
            }
        ),
    },
}


# Send POST request to the API endpoint
response = requests.post(f"http://localhost:{port}/generate", json=payload)
print_highlight(response.json())

[2025-09-01 06:38:10] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 22, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-09-01 06:38:10] Decode batch. #running-req: 1, #token: 41, token usage: 0.00, cuda graph: True, gen throughput (token/s): 100.23, #queue-req: 0, 


[2025-09-01 06:38:11] Decode batch. #running-req: 1, #token: 81, token usage: 0.00, cuda graph: True, gen throughput (token/s): 106.28, #queue-req: 0, 


[2025-09-01 06:38:11] Decode batch. #running-req: 1, #token: 121, token usage: 0.01, cuda graph: True, gen throughput (token/s): 106.36, #queue-req: 0, 


[2025-09-01 06:38:11] Decode batch. #running-req: 1, #token: 161, token usage: 0.01, cuda graph: True, gen throughput (token/s): 133.91, #queue-req: 0, 


[2025-09-01 06:38:12] Decode batch. #running-req: 1, #token: 201, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.85, #queue-req: 0, 


...

[2025-09-01 06:38:15] Decode batch. #running-req: 1, #token: 721, token usage: 0.04, cuda graph: True, gen throughput (token/s): 164.11, #queue-req: 0, 


[2025-09-01 06:38:15] Decode batch. #running-req: 1, #token: 761, token usage: 0.04, cuda graph: True, gen throughput (token/s): 161.48, #queue-req: 0, 


[2025-09-01 06:38:15] Decode batch. #running-req: 1, #token: 801, token usage: 0.04, cuda graph: True, gen throughput (token/s): 160.98, #queue-req: 0, 


[2025-09-01 06:38:16] Decode batch. #running-req: 1, #token: 841, token usage: 0.04, cuda graph: True, gen throughput (token/s): 161.76, #queue-req: 0, 


[2025-09-01 06:38:16] Decode batch. #running-req: 1, #token: 881, token usage: 0.04, cuda graph: True, gen throughput (token/s): 163.97, #queue-req: 0, 
[2025-09-01 06:38:16] INFO:     127.0.0.1:56044 - "POST /generate HTTP/1.1" 200 OK


In [12]:
terminate_process(server_process)

## Offline Engine API

In [13]:
import sglang as sgl

llm = sgl.Engine(
    model_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    reasoning_parser="deepseek-r1",
    grammar_backend="xgrammar",
)

W0901 06:38:18.149000 2827546 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0901 06:38:18.149000 2827546 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


All deep_gemm operations loaded successfully!


`torch_dtype` is deprecated! Use `dtype` instead!


W0901 06:38:27.004000 2830078 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0901 06:38:27.004000 2830078 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


`torch_dtype` is deprecated! Use `dtype` instead!
[2025-09-01 06:38:27] `torch_dtype` is deprecated! Use `dtype` instead!


...

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.30s/it]


Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.24s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.25s/it]



Capturing batches (bs=4 avail_mem=62.72 GB):   0%|          | 0/3 [00:00<?, ?it/s]

Capturing batches (bs=1 avail_mem=62.65 GB): 100%|██████████| 3/3 [00:00<00:00,  9.95it/s]


### JSON

**Using Pydantic**

In [14]:
import json
from pydantic import BaseModel, Field


prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


sampling_params = {
    "temperature": 0,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "json_schema": json.dumps(CapitalInfo.model_json_schema()),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugou",
    "Tibetan",
    "Hui",
    "Mongolian"
  ],
  "KeySights": [
    "The Great Wall of China",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need anything else!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude"

**JSON Schema Directly**

In [15]:
prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

sampling_params = {"temperature": 0, "max_new_tokens": 2048, "json_schema": json_schema}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugou",
    "Tibetan",
    "Hui",
    "Mongolian"
  ],
  "KeySights": [
    "The Great Wall of China",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need anything else!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude"

### EBNF


In [16]:
prompts = [
    "Give me the information of the capital of France.",
    "Give me the information of the capital of Germany.",
    "Give me the information of the capital of Italy.",
]

sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "ebnf": (
        "root ::= city | description\n"
        'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
        'description ::= city " is " status\n'
        'status ::= "the capital of " country\n'
        'country ::= "England" | "France" | "Germany" | "Italy"'
    ),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of France.
Generated text: 
Alright, so I need to figure out the capital of France. I remember that capitals are the main cities where countries are declared to be governed, right? France is a country I'm somewhat familiar with, but I'm not entirely sure about its capital. Let me think...

First, I know that Paris is a major city in France. It's famous for landmarks like the Eiffel Tower, the Louvre Museum, and Notre-Dame. These are all well-known attractions, so maybe that's the capital? But I'm not 100% certain. 

Wait, I think I've heard somewhere that Paris is
Prompt: Give me the information of the capital of Germany.
Generated text: 
The capital of Germany is Berlin.

That's it.

Alright, so I need to explain the information about the capital of Germany. Hmm, I know that the capital is Berlin, but I should probably go into more detail to make it informative. Maybe I can talk about why Berlin is the capital, its significance, history, 

### Regular expression

In [17]:
prompts = [
    "Please provide information about London as a major global city:",
    "Please provide information about Paris as a major global city:",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95, "regex": "(France|England)"}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Please provide information about London as a major global city:
Generated text:  its history, culture, and economy.700-800 words.

Please also include information about the global economic and political impact of COVID-19 on London.
</think>France
Prompt: Please provide information about Paris as a major global city:
Generated text:  its location, historical significance, current status, and future projections.
 For the current status and future projections, please provide data from the year 2022 onwards.
 Paris is a major global city known for its rich history, cultural significance, and vibrant modern lifestyle. It has been a capital of France since the Middle Ages and has played a central role in European history. The city is located in western Europe, bordered by the Seine River to the west and the Eure and Seine rivers to the east. Paris is home to a diverse population of over 2.1 million people, making it one of the most populous urban


In [18]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
prompts = [text]


sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "structural_tag": json.dumps(
        {
            "type": "structural_tag",
            "structures": [
                {
                    "begin": "<function=get_current_weather>",
                    "schema": schema_get_current_weather,
                    "end": "</function>",
                },
                {
                    "begin": "<function=get_current_date>",
                    "schema": schema_get_current_date,
                    "end": "</function>",
                },
            ],
            "triggers": ["<function="],
        }
    ),
}


# Send POST request to the API endpoint
outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: <｜begin▁of▁sentence｜><｜Assistant｜>Give me the information and population of the capital of France in the JSON format.<｜end▁of▁sentence｜><｜Assistant｜><think>

Generated text: Alright, the user is asking for the information and population of the capital of France in JSON format. First, I need to identify what exactly they're looking for. The capital of France is Paris, so that's straightforward.

Next, I should figure out what details they need. They mentioned "information and population," so I should include both demographic data and perhaps some key facts about the city. Demographics usually include population numbers, age distribution, and maybe the number of residents versus visitors.

I should also think about where the information is reliable. The latest data from the National Institute of Statistics and Administrative Services (INSEE) in France would be accurate. I remember that as of recent years, Paris has had varying population numbers, so I should note that the figure 

In [19]:
llm.shutdown()